![](https://miro.medium.com/max/1838/1*BHzGVskWGS_3jEcYYi6miQ.png)

https://youtu.be/U0s0f995w14

In [2]:
import torch
import torch.nn as nn

In [4]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super().__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        assert (self.head_dim * heads == embed_size) # make sure embed_size is divisible by heads
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        
    def forward(self, values, keys, queries, mask):
        N = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]
        
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)
        
        # QK^T
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # query shape:  (N, query_len, heads, heads_dim) -> (n,q,h,d)
        # keys shape:   (N, key_len, heads, heads_dim)   -> (n,k,h,d)
        # energy shape: (N, heads, query_len, key_len)   -> (n,h,q,k)
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float('-1e20'))
            
        QKT_sqrtd = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)
        
        # softmax(...) * V
        attention = torch.einsum("nhql,nlhd->nqhd", [QKT_sqrtd, values])
        # key_len == value_len
        # QKT_sqrtd shape = energy shape: (N, heads, query_len, key_len)   -> (n,h,q,k)
        # values shape:                   (N, value_len, heads, heads_dim) -> (n,v,h,d)
        # attention shape:                (N, query_len, heads, head_dim)  -> (n,q,h,d)
        
        # concatenation
        attention = attention.reshape(N, query_len, self.heads*self.head_dim)
        
        return self.fc_out(attention)

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super().__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(sefl, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        
        x = self.norm1(attention + query)
        x = self.dropout(x)
        
        forward = self.feed_forward(x)
        
        x = self.norm2(forward + x)
        x = self.dropout(x)
        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
        super().__init__()
        self.embed_size = embed_size
